# Hybrid Plugin System

This notebook illustrates usage of the Hybrid Plugin System. We combine a Python manager plugin and a C++ manager plugin, and dispatch to the appropriate plugin based on its capabilities and priority. 

The primary use-case for this feature is to allow performance critical functionality to be written in performant C++, whilst less performance critical functionality can be written in more flexible Python.

The hybrid plugin system also provides a more convenient abstraction for working with multiple plugin systems in general. It should be the default choice by most host applications (i.e. where a Python environment is available).

## How it works

The hybrid plugin system allows a manager to split its implementation between multiple languages. Under the hood, each language is loaded using its own plugin system (currently, C++ and Python are supported). It combines plugins that share the same identifier so they can be used by a host as if they were a single implementation. Calls are routed to one of the underlying implementations based on capabilities and the language priority set when the factory is made.

This allows high call count methods such as `resolve` to be implemented in C++ to ensure maximum performance, whilst less frequently used methods such as `register` remain in more flexible Python.

The following subsection dives into some more detail of how this works.

### Details

OpenAssetIO manager plugins must advertise a unique identifier. An OpenAssetIO plugin system (e.g. C++ or Python) maps unique identifiers to a plugin instance. A consequence of this is if multiple plugins advertise the same unique identifier, only one of those plugins can be chosen _by that plugin system_. However, if multiple plugin systems are in use,
then each plugin system has their own mapping of unique identifier to plugin instance. This means there _can_ be multiple plugins with the same identifier, as long as they are discovered by different plugin systems.

This is the essence of how the hybrid plugin system discovers plugins. If two (or more) plugins from two (or more) different plugin systems advertise the same unique identifier, then we assume they are related and can be composed.

The hybrid plugin system therefore wraps a list of child plugin systems, such that they present as a single plugin system to the host.

Given that the hybrid plugin system has discovered two (or more) composable plugins, we then need a mechanism to dispatch API calls to the appropriate plugin. I.e. we need a way to choose which plugin is the best to use for a particular API call. This is where the `hasCapability` API method comes in.

OpenAssetIO API methods are grouped under "capabilities", e.g. `"resolution"`, `"publishing"`, `"relationshipQueries"`, etc (these are stringified representations of the `Capability` enumeration). A manager plugin advertises which capabilities it supports by overriding the `hasCapability` method of the base `ManagerInterface` class.

Therefore, we can dispatch an API call to the appropriate manager plugin by finding the plugin that advertises the associated capability for that API call.

If multiple plugins advertise that they support the required capability, then which plugin to use is determined by the original order that the child plugin systems were provided to the hybrid plugin system.  For example, if the hybrid plugin system was constructed with a list containing the C++ plugin system followed by the Python plugin system, and C++ and Python plugins have been discovered and composed, and both advertise that they support the required capability for a particular API call, then the C++ plugin will be chosen for that API call.

All the OpenAssetIO _required_ capabilities (i.e. `"entityReferenceIdentification"`, `"managementPolicyQueries"`, `"entityTraitIntrospection"`) must be satisfied by at least one of the composed plugins.

If only one child factory locates a plugin with the desired identifier, then that plugin is used directly (i.e. the plugin is not wrapped). In this way, host applications making use of the hybrid plugin system don't lose out on any functionality or performance. 

#### A note on information sharing

Having two plugins in completely different languages, which logically form a single plugin, raises the question of how data should be shared between them.

OpenAssetIO has a mechanism to help with this via the `Context` object. A `Context` instance is passed to (almost) every API method. A well-behaved host will re-use the same `Context` for all requests in the same logical process (typically an application session). 

The `Context` object holds a `managerState`  object, which can be used to communicate arbitrary information between the plugins. See the [API documentation](http://docs.openassetio.org/OpenAssetIO/stable_resolution.html#stable_resolution_manager_state) for more information.

Populating the manager state in a way that can be read by both Python and C++ is left as an exercise to the reader. It's likely that the C++ plugin will require CPython as a dependency in order to translate between languages.

## Example

### Preamble

Reading through the following is not necessary to be able to understand how to use the hybrid plugin system, and can be safely skipped. Let's get the standard OpenAssetIO bootstrapping boilerplate out of the way. See the "Hello OpenAssetIO" notebook for more details.

In [1]:

try:
    import openassetio
    import openassetio_mediacreation
except ImportError:
    print(
        "This notebook requires the packages listed in `resources/requirements.txt` to be installed")
    raise

from resources import helpers

from openassetio.hostApi import HostInterface, ManagerFactory
from openassetio.log import LoggerInterface


class NotebookHostInterface(HostInterface):
    def identifier(self):
        return "org.jupyter.notebook"

    def displayName(self):
        return "Jupyter Notebook"


class NullLogger(LoggerInterface):
    def log(self, _severity, _message):
        pass


host_interface = NotebookHostInterface()

logger = NullLogger()

#### The example plugin(s)


In order to illustrate the hybrid plugin system, we'll make use of a super simple example hybrid plugin created just for this notebook, available in `resources/hybrid_plugin_system/SimpleHybridManager`.

The Python plugin component is trivially available. However, the C++ plugin component is more complex, and must be built with a compiler toolchain compatible with the OpenAssetIO libraries in the Python environment of this notebook. See `resources/hybrid_plugin_system/SimpleHybridManager/README.md` for more details. 

We assume both the C++ and Python plugin components are installed into `resources/hybrid_plugin_system/SimpleHybridManager/plugin`, and will be discovered by adding this location to the standard `OPENASSETIO_PLUGIN_PATH` environment variable.

In [2]:
import os


os.environ["OPENASSETIO_PLUGIN_PATH"] = os.path.join(
    "resources", "hybrid_plugin_system", "SimpleHybridManager", "plugin")

### Plugins designed for composition

Let's try to initialise our C++ and Python example managers separately and see what happens.


In [3]:
from openassetio.errors import ConfigurationException
from openassetio.pluginSystem import (
    CppPluginSystemManagerImplementationFactory, PythonPluginSystemManagerImplementationFactory)


cpp_factory = CppPluginSystemManagerImplementationFactory(logger)

try:
    cpp_manager = ManagerFactory.defaultManagerForInterface(
        "resources/hybrid_plugin_system/openassetio_config.toml",
        host_interface,
        cpp_factory,
        logger)

except ConfigurationException as exc:
    helpers.display_result(f"C++ plugin error: {exc}")

python_factory = PythonPluginSystemManagerImplementationFactory(logger)

py_manager = ManagerFactory.defaultManagerForInterface(
    "resources/hybrid_plugin_system/openassetio_config.toml",
    host_interface,
    python_factory,
    logger)

> **Result:**
> `C++ plugin error: Manager implementation for 'org.openassetio.examples.simplehybridmanager' does not support the required capabilities: entityReferenceIdentification, managementPolicyQueries, entityTraitIntrospection`

So the C++ plugin system found a plugin, but it doesn't support any of the required capabilities.

We had better luck with the Python plugin system. However, we're going to want to `resolve` an entity. Is the Python plugin capable of resolution?

In [4]:
can_resolve = py_manager.hasCapability(py_manager.Capability.kResolution)

helpers.display_result(f"Can resolve? {can_resolve}")

> **Result:**
> `Can resolve? False`

So no, the Python plugin does not support the `resolve` method, at least on its own...

### The hybrid plugin system

Given the two `ManagerImplementationFactoryInterface` instances (`cpp_factory` and `python_factory`), we can create a hybrid factory.

In [5]:
from openassetio.pluginSystem import HybridPluginSystemManagerImplementationFactory


hybrid_factory = HybridPluginSystemManagerImplementationFactory(
    [cpp_factory, python_factory], logger)

manager = ManagerFactory.defaultManagerForInterface(
    "resources/hybrid_plugin_system/openassetio_config.toml",
    host_interface,
    hybrid_factory,
    logger)

Success! 

Notice how only a single config file (`openassetio_config.toml`) was provided. With hybrid plugins, the same configuration file is used for all the constituent plugins. In particular, any and all manager settings specified in the config file are passed to all plugins during initialisation.

Is this combined hybrid plugin now capable of resolution?

In [6]:
can_resolve = manager.hasCapability(py_manager.Capability.kResolution)

helpers.display_result(f"Can resolve? {can_resolve}")

> **Result:**
> `Can resolve? True`

Again, success! The resolution capability from the C++ plugin has been combined with the capabilities of the Python plugin. 

Note that since the `resolve` method is implemented in the C++ plugin, the Python GIL will be released when calling this method, allowing Python threads to continue whilst the `resolve` call is processed. In particular, since many UIs are written in Python, allowing (multiple/batch) `resolve` calls to run in a separate thread, without holding the Python GIL, can prevent nasty UI lockups.


Now lets retrieve an entity's trait set from the manager:

In [7]:
from openassetio.access import EntityTraitsAccess


context = manager.createContext()
entity_ref = manager.createEntityReference("examplehybrid://example_entity")

trait_set = manager.entityTraits(entity_ref, EntityTraitsAccess.kRead, context)

helpers.display_result(trait_set)


> **Result:**
> `{'openassetio-mediacreation:usage.Entity', 'openassetio-mediacreation:content.LocatableContent'}`

So the entity has the `LocatableContent` trait. Let's `resolve` its location:


In [8]:
from openassetio.access import ResolveAccess
from openassetio_mediacreation.traits.content import LocatableContentTrait


trait_data = manager.resolve(entity_ref, {LocatableContentTrait.kId}, ResolveAccess.kRead, context)

url = LocatableContentTrait(trait_data).getLocation()

helpers.display_result(url)


> **Result:**
> `file:///some/path.exr`

Success! Pretty straightforward.
 
For further reading, if you inspect the C++ implementation at `resources/hybrid_plugin_system/SimpleHybridManager/src/CppComponentOfSimpleHybridManager.cpp` you'll find no implementation of the `entityTraits` method, or indeed any non-trivial method, other than `resolve`. 

Similarly, if you inspect the Python implementation at `resources/hybrid_plugin_system/SimpleHybridManager/plugin/PyComponentOfSimpleHybridManager.py` you'll find no implementation of the `resolve` method. 

So the two plugins have been seamlessly combined into a single interface.